# SQL query from table names - Continued

In [1]:
!pip install python-dotenv

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [7]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

I see you have not provided any tables for me to work with. Could you please provide the tables so that I can assist you with your queries?


In [8]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

SELECT * 
FROM employees;
```
This SQL query selects all data from the "employees" table.


In [9]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

I see that you have not provided any tables for me to work with. Could you please provide the tables so that I can assist you with your queries?


In [10]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

Please provide a specific query to assist you.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [11]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# Load environment variables
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Function to query OpenAI with context and user question
def return_CCRMSQL(user_message, context, model="gpt-3.5-turbo", temperature=0):
    client = OpenAI(api_key=OPENAI_API_KEY)

    new_context = context.copy()
    new_context.append({'role': 'user', 'content': f"question: {user_message}"})

    response = client.chat.completions.create(
        model=model,
        messages=new_context,
        temperature=temperature
    )

    return response.choices[0].message.content

# Context: restaurant sales database with tables and sample data
context = [{'role': 'system', 'content': """
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    name VARCHAR(100),
    phone VARCHAR(20)
);

CREATE TABLE menu_items (
    item_id INTEGER PRIMARY KEY,
    name VARCHAR(100),
    category VARCHAR(50),
    price FLOAT
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    item_id INTEGER,
    quantity INTEGER,
    order_date DATE
);

-- Sample Data
INSERT INTO customers VALUES (1, 'Sarah', '123-456'), (2, 'Ali', '789-101'), (3, 'Lina', '202-303');
INSERT INTO menu_items VALUES (1, 'Burger', 'Main', 8.5), (2, 'Fries', 'Side', 3.0), (3, 'Cola', 'Drink', 2.0);
INSERT INTO orders VALUES (1001, 1, 1, 2, '2024-03-01'), (1002, 1, 2, 1, '2024-03-01'), (1003, 2, 3, 3, '2024-03-02');

-- Example Queries
-- Q: What are the menu items?
-- SQL: SELECT name FROM menu_items;

-- Q: How many times was each item ordered?
-- SQL: SELECT name, SUM(quantity) AS total_sold FROM menu_items m JOIN orders o ON m.item_id = o.item_id GROUP BY name;

-- Q: What did customer Sarah order?
-- SQL: SELECT m.name, o.quantity FROM orders o JOIN menu_items m ON o.item_id = m.item_id JOIN customers c ON o.customer_id = c.customer_id WHERE c.name = 'Sarah';
"""}]

# Test queries
print(return_CCRMSQL("List all menu item names and their prices.", context))
print(return_CCRMSQL("How many times was each item ordered?", context))
print(return_CCRMSQL("Show orders placed by customer Sarah.", context))


```sql
SELECT name, price
FROM menu_items;
```
```sql
SELECT name, SUM(quantity) AS total_sold 
FROM menu_items m 
JOIN orders o ON m.item_id = o.item_id 
GROUP BY name;
```
```sql
SELECT m.name AS item_name, o.quantity, o.order_date
FROM orders o
JOIN menu_items m ON o.item_id = m.item_id
JOIN customers c ON o.customer_id = c.customer_id
WHERE c.name = 'Sarah';
```


# Restaurant Sales.


### Key Findings
-Structured prompts that include CREATE TABLE, sample inserts, and example queries improved GPT's accuracy significantly.

-Few-shot examples helped guide GPT in understanding expected syntax and reduced hallucinations.

-Adding descriptive table names and meaningful field names (e.g., order_date, customer_id) made a difference in output clarity.

### That Didn't Work Well
-When context lacked clear relationships between tables (e.g., missing foreign key-like links), GPT made assumptions that were incorrect.

-Ambiguous or overly short user questions led to vague or incorrect SQL queries.

-In some tests, GPT hallucinated column names not present in the schema when few-shot examples were missing.

### What I Learned
-Prompt engineering matters a lot. The more structured and concrete the context, the better GPT performs.

-Using SQL-style schema with actual data examples is much better than plain English descriptions.

-Few-shot learning is powerful—even 2-3 good examples greatly increase output reliability.

-GPT can be a solid tool for text-to-SQL translation, but guardrails are needed to catch edge cases or wrong assumptions.